In [ ]:
import pandas as pd
import os
from db.connection import DBConnection  # Importa a classe de conexão com o banco de dados

print("Pacotes instalados com sucesso!")


Pacotes instalados com sucesso!


In [ ]:
# Carregar o arquivo Excel com as linhas irrelevantes ignoradas
file_path = "data/CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE.xlsx"
df = pd.read_excel(file_path, sheet_name="CONSUMO POR UF", skiprows=4)

# Renomear a primeira coluna para "Estado"
df.rename(columns={df.columns[0]: "Estado"}, inplace=True)

# Excluir a linha de totalização
df = df[df["Estado"] != "TOTAL POR UF"]

df.head()




,Estado,2004,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252
0,NaN,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,...,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
2,Rondônia,112812,105806,106668,107580,112115.8,107816.5,108589,110858,116058,...,326466.34,338370.94,349553.32,325289.48,336286.08,341360.74,NaN,NaN,NaN,NaN
3,Acre,34840.05,33050.33,33858.03,34961.02,29766.47,32048.6,33659.7,34837.46,36879.8,...,104085.49,114835.17,113443.06,101217.72,103114.79,109270.7,NaN,NaN,NaN,NaN
4,Amazonas,274773,278429.72,270420,279044.7,283866,285251,295884,296649.53,308616.48,...,639770,638760,653774,636390,686657,723105,NaN,NaN,NaN,NaN
5,Roraima,31695.63,30802.28,30275.52,33035.03,28944.36,29531.8,28212.49,28722.61,29796.91,...,120493.2,127774.23,110801.21,92165.76,102722.42,110772.75,NaN,NaN,NaN,NaN


In [34]:
# Separar os anos e meses dos cabeçalhos mesclados
anos = []
meses = df.columns[1:]  # Colunas a partir da segunda
for col in meses:
    if isinstance(col, str) and "Unnamed" in col:
        anos.append(anos[-1])  # Repete o ano anterior
    else:
        anos.append(col)  # Novo ano encontrado

# Corrigir os meses (segunda linha do cabeçalho)
meses = df.iloc[0, 1:].tolist()  # A segunda linha são os meses
df = df[1:]  # Remover a linha de meses

# Atualizar as colunas com o novo cabeçalho (Ano e Mês)
df.columns = ["Estado"] + [f"{ano}_{mes}" for ano, mes in zip(anos, meses)]

# Transformar os dados em formato longo (Estado, Ano_Mês, Consumo)
df_long = df.melt(id_vars=["Estado"], var_name="Ano_Mês", value_name="Consumo_MWh")

# Separar Ano e Mês
df_long["Ano"] = df_long["Ano_Mês"].str.extract(r"(\d{4})")
df_long["Mês"] = df_long["Ano_Mês"].str.extract(r"([A-Z]{3})")
df_long.drop(columns=["Ano_Mês"], inplace=True)

# Converter Consumo para numérico
df_long["Consumo_MWh"] = pd.to_numeric(df_long["Consumo_MWh"], errors="coerce")

df_long.head()

,Estado,Consumo_MWh,Ano,Mês
0,Rondônia,112812.00,2004,JAN
1,Acre,34840.05,2004,JAN
2,Amazonas,274773.00,2004,JAN
3,Roraima,31695.63,2004,JAN
4,Pará,1011353.04,2004,JAN


In [35]:
# Remover valores ausentes
df_long = df_long.dropna()

# Adicionar códigos de estado (mapeamento manual)
estado_codigos = {
    "Rondônia": 11, "Acre": 12, "Amazonas": 13, "Roraima": 14, "Pará": 15, "Amapá": 16, "Tocantins": 17,
    "Maranhão": 21, "Piauí": 22, "Ceará": 23, "Rio Grande do Norte": 24, "Paraíba": 25, "Pernambuco": 26, "Alagoas": 27, "Sergipe": 28, "Bahia": 29,
    "Minas Gerais": 31, "Espírito Santo": 32, "Rio de Janeiro": 33, "São Paulo": 35,
    "Paraná": 41, "Santa Catarina": 42, "Rio Grande do Sul": 43,
    "Mato Grosso do Sul": 50, "Mato Grosso": 51, "Goiás": 52, "Distrito Federal": 53  
}
df_long["cd_estado"] = df_long["Estado"].map(estado_codigos)

# Remover linhas com estados desconhecidos
df_long = df_long.dropna(subset=["cd_estado"])

# Mapeamento de meses para códigos
meses_codigos = {"JAN": 1, "FEV": 2, "MAR": 3, "ABR": 4, "MAI": 5, "JUN": 6,
                 "JUL": 7, "AGO": 8, "SET": 9, "OUT": 10, "NOV": 11, "DEZ": 12}
df_long["cd_mes"] = df_long["Mês"].map(meses_codigos)


In [25]:
df_long.head()

,Estado,Consumo_MWh,Ano,Mês,cd_estado,cd_mes
0,Rondônia,112812.00,2004,JAN,4.0,1
1,Acre,34840.05,2004,JAN,1.0,1
2,Amazonas,274773.00,2004,JAN,2.0,1
3,Roraima,31695.63,2004,JAN,5.0,1
4,Pará,1011353.04,2004,JAN,3.0,1


In [36]:
# Inserir os dados no banco usando a classe DBConnection
with DBConnection() as conn:
    cursor = conn.cursor()
    for _, row in df_long.iterrows():
        cursor.execute("""
            INSERT INTO t_sem_consumo_energia (cd_estado, cd_ano, cd_mes, vl_consumo)
            VALUES (:1, :2, :3, :4)
        """, (
            int(row["cd_estado"]), int(row["Ano"]), int(row["cd_mes"]), row["Consumo_MWh"]
        ))
    conn.commit()

print("Dados importados com sucesso!")

Dados importados com sucesso!
